# All Necessary Imports

In [1]:
import pandas as pd
import numpy as np

In [2]:
import category_encoders as ce
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
# Environment settings
pd.options.display.max_columns = 99

# Read the inputs from different input data files and peep into the contents of dataframe

In [4]:
## Adult Income Data and Telecom Churn Data

In [5]:
# The code was removed by Watson Studio for sharing.

In [6]:
# The code was removed by Watson Studio for sharing.

# Exploratory Data Analysis

In [7]:
## Adult Data

In [8]:
dfAdultData.shape

(48842, 15)

In [9]:
dfAdultData.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [10]:
NumericFeatures=dfAdultData.select_dtypes(include=['int64', 'float64']).columns
NumericFeatures

Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object')

In [11]:
dfAdultData[NumericFeatures].isna().sum()

age                0
fnlwgt             0
educational-num    0
capital-gain       0
capital-loss       0
hours-per-week     0
dtype: int64

### No numeric column is empty

In [12]:
CategoricFeatures=dfAdultData.select_dtypes(include=['object']).columns
CategoricFeatures

Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'gender', 'native-country', 'income'],
      dtype='object')

In [13]:
dfAdultData[CategoricFeatures].isna().sum()

workclass         0
education         0
marital-status    0
occupation        0
relationship      0
race              0
gender            0
native-country    0
income            0
dtype: int64

In [14]:
for cate in CategoricFeatures :
    df = dfAdultData[cate].value_counts(dropna=False).to_frame()
    print(df)

                  workclass
Private               33906
Self-emp-not-inc       3862
Local-gov              3136
?                      2799
State-gov              1981
Self-emp-inc           1695
Federal-gov            1432
Without-pay              21
Never-worked             10
              education
HS-grad           15784
Some-college      10878
Bachelors          8025
Masters            2657
Assoc-voc          2061
11th               1812
Assoc-acdm         1601
10th               1389
7th-8th             955
Prof-school         834
9th                 756
12th                657
Doctorate           594
5th-6th             509
1st-4th             247
Preschool            83
                       marital-status
Married-civ-spouse              22379
Never-married                   16117
Divorced                         6633
Separated                        1530
Widowed                          1518
Married-spouse-absent             628
Married-AF-spouse                  37
        

<H4> There are pseudo NaNs in columns in the form of "?" Need to see how to handle these.
    Since the number of rows are high (48842), we might as well as drop these rows containing ?

In [15]:
ColumnDict = {}

In [16]:

ColumnDict = {'Adult' : {'Numeric': NumericFeatures, 'Categoric': CategoricFeatures}}

In [17]:
ColumnDict

{'Adult': {'Numeric': Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
         'hours-per-week'],
        dtype='object'),
  'Categoric': Index(['workclass', 'education', 'marital-status', 'occupation',
         'relationship', 'race', 'gender', 'native-country', 'income'],
        dtype='object')}}

In [18]:
## Common Routine
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
        html_str+="<td>&nbsp&nbsp&nbsp&nbsp</td>"
    #print(html_str)
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [19]:
display_side_by_side (dfAdultData['workclass'].value_counts(dropna=False).to_frame(), \
                      dfAdultData['education'].value_counts(dropna=False).to_frame(), \
                      dfAdultData['marital-status'].value_counts(dropna=False).to_frame(), \
                      dfAdultData['occupation'].value_counts(dropna=False).to_frame(), \
                      dfAdultData['race'].value_counts(dropna=False).to_frame(), \
                      dfAdultData['native-country'].value_counts(dropna=False).to_frame()
                     )

workclass 
 
 
 
 
 Private 
 33906 
 
 
 Self-emp-not-inc 
 3862 
 
 
 Local-gov 
 3136 
 
 
 ? 
 2799 
 
 
 State-gov 
 1981 
 
 
 Self-emp-inc 
 1695 
 
 
 Federal-gov 
 1432 
 
 
 Without-pay 
 21 
 
 
 Never-worked 
 10 
 
 
      
 
 
 
 education 
 
 
 
 
 HS-grad 
 15784 
 
 
 Some-college 
 10878 
 
 
 Bachelors 
 8025 
 
 
 Masters 
 2657 
 
 
 Assoc-voc 
 2061 
 
 
 11th 
 1812 
 
 
 Assoc-acdm 
 1601 
 
 
 10th 
 1389 
 
 
 7th-8th 
 955 
 
 
 Prof-school 
 834 
 
 
 9th 
 756 
 
 
 12th 
 657 
 
 
 Doctorate 
 594 
 
 
 5th-6th 
 509 
 
 
 1st-4th 
 247 
 
 
 Preschool 
 83 
 
 
      
 
 
 
 marital-status 
 
 
 
 
 Married-civ-spouse 
 22379 
 
 
 Never-married 
 16117 
 
 
 Divorced 
 6633 
 
 
 Separated 
 1530 
 
 
 Widowed 
 1518 
 
 
 Married-spouse-absent 
 628 
 
 
 Married-AF-spouse 
 37 
 
 
      
 
 
 
 occupation 
 
 
 
 
 Prof-specialty 
 6172 
 
 
 Craft-repair 
 6112 
 
 
 Exec-managerial 
 6086 
 
 
 Adm-clerical 
 5611 
 
 
 Sales 
 5504 
 
 
 Other-service 
 4923 
 
 
 Machine-op-inspct 
 3022 
 
 
 ? 
 2809 
 
 
 Transport-moving 
 2355 
 
 
 Handlers-cleaners 
 2072 
 
 
 Farming-fishing 
 1490 
 
 
 Tech-support 
 1446 
 
 
 Protective-serv 
 983 
 
 
 Priv-house-serv 
 242 
 
 
 Armed-Forces 
 15 
 
 
      
 
 
 
 race 
 
 
 
 
 White 
 41762 
 
 
 Black 
 4685 
 
 
 Asian-Pac-Islander 
 1519 
 
 
 Amer-Indian-Eskimo 
 470 
 
 
 Other 
 406 
 
 
      
 
 
 
 native-country 
 
 
 
 
 United-States 
 43832 
 
 
 Mexico 
 951 
 
 
 ? 
 857 
 
 
 Philippines 
 295 
 
 
 Germany 
 206 
 
 
 Puerto-Rico 
 184 
 
 
 Canada 
 182 
 
 
 El-Salvador 
 155 
 
 
 India 
 151 
 
 
 Cuba 
 138 
 
 
 England 
 127 
 
 
 China 
 122 
 
 
 South 
 115 
 
 
 Jamaica 
 106 
 
 
 Italy 
 105 
 
 
 Dominican-Republic 
 103 
 
 
 Japan 
 92 
 
 
 Guatemala 
 88 
 
 
 Poland 
 87 
 
 
 Vietnam 
 86 
 
 
 Columbia 
 85 
 
 
 Haiti 
 75 
 
 
 Portugal 
 67 
 
 
 Taiwan 
 65 
 
 
 Iran 
 59 
 
 
 Greece 
 49 
 
 
 Nicaragua 
 49 
 
 
 Peru 
 46 
 
 
 Ecuador 
 45 
 
 
 France 
 38 
 
 
 Ireland 
 37 
 
 
 Thailand 
 30 
 
 
 Hong 
 30 
 
 
 Cambodia 
 28 
 
 
 Trinadad&Tobago 
 27 
 
 
 Yugoslavia 
 23 
 
 
 Outlying-US(Guam-USVI-etc) 
 23 
 
 
 Laos 
 23 
 
 
 Scotland 
 21 
 
 
 Honduras 
 20 
 
 
 Hungary 
 19 
 
 
 Holand-Netherlands 
 1

### Clearly : Native-country is not a candidate for one-hot encoding. <br>Occupation should also be avoided for one-hot encoding. The value coming in the test data-set might have new categories.

## Telecom Data

In [20]:
NumericFeatures = dfTelcoChurn.select_dtypes(include=['int64', 'float64']).columns
NumericFeatures

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges'], dtype='object')

In [21]:
CategoricalFeatures = dfTelcoChurn.select_dtypes(include='object').columns

In [22]:
dfTelcoChurn[NumericFeatures].isna().sum()

SeniorCitizen     0
tenure            0
MonthlyCharges    0
dtype: int64

In [23]:
dfTelcoChurn[CategoricalFeatures].isna().sum()

customerID          0
gender              0
Partner             0
Dependents          0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
TotalCharges        0
Churn               0
dtype: int64

In [24]:
display_side_by_side(dfTelcoChurn['MultipleLines'].value_counts(dropna=False).to_frame(), \
                    dfTelcoChurn['InternetService'].value_counts(dropna=False).to_frame(), \
                    dfTelcoChurn['OnlineSecurity'].value_counts(dropna=False).to_frame() , \
                    dfTelcoChurn['OnlineBackup'].value_counts(dropna=False).to_frame() ,\
                    dfTelcoChurn['DeviceProtection'].value_counts(dropna=False).to_frame())

MultipleLines 
 
 
 
 
 No 
 3390 
 
 
 Yes 
 2971 
 
 
 No phone service 
 682 
 
 
      
 
 
 
 InternetService 
 
 
 
 
 Fiber optic 
 3096 
 
 
 DSL 
 2421 
 
 
 No 
 1526 
 
 
      
 
 
 
 OnlineSecurity 
 
 
 
 
 No 
 3498 
 
 
 Yes 
 2019 
 
 
 No internet service 
 1526 
 
 
      
 
 
 
 OnlineBackup 
 
 
 
 
 No 
 3088 
 
 
 Yes 
 2429 
 
 
 No internet service 
 1526 
 
 
      
 
 
 
 DeviceProtection 
 
 
 
 
 No 
 3095 
 
 
 Yes 
 2422 
 
 
 No internet service 
 1526

In [25]:
display_side_by_side(dfTelcoChurn['TechSupport'].value_counts(dropna=False).to_frame(), \
                    dfTelcoChurn['StreamingTV'].value_counts(dropna=False).to_frame(), \
                    dfTelcoChurn['StreamingMovies'].value_counts(dropna=False).to_frame() , \
                    dfTelcoChurn['Contract'].value_counts(dropna=False).to_frame() ,\
                    dfTelcoChurn['PaperlessBilling'].value_counts(dropna=False).to_frame())

TechSupport 
 
 
 
 
 No 
 3473 
 
 
 Yes 
 2044 
 
 
 No internet service 
 1526 
 
 
      
 
 
 
 StreamingTV 
 
 
 
 
 No 
 2810 
 
 
 Yes 
 2707 
 
 
 No internet service 
 1526 
 
 
      
 
 
 
 StreamingMovies 
 
 
 
 
 No 
 2785 
 
 
 Yes 
 2732 
 
 
 No internet service 
 1526 
 
 
      
 
 
 
 Contract 
 
 
 
 
 Month-to-month 
 3875 
 
 
 Two year 
 1695 
 
 
 One year 
 1473 
 
 
      
 
 
 
 PaperlessBilling 
 
 
 
 
 Yes 
 4171 
 
 
 No 
 2872

In [26]:
display_side_by_side(dfTelcoChurn['PaymentMethod'].value_counts(dropna=False).to_frame(), \
                    dfTelcoChurn['TotalCharges'].value_counts(dropna=False).to_frame() )

PaymentMethod 
 
 
 
 
 Electronic check 
 2365 
 
 
 Mailed check 
 1612 
 
 
 Bank transfer (automatic) 
 1544 
 
 
 Credit card (automatic) 
 1522 
 
 
      
 
 
 
 TotalCharges 
 
 
 
 
 
 11 
 
 
 20.2 
 11 
 
 
 19.75 
 9 
 
 
 20.05 
 8 
 
 
 19.65 
 8 
 
 
 19.9 
 8 
 
 
 19.55 
 7 
 
 
 45.3 
 7 
 
 
 20.25 
 6 
 
 
 20.15 
 6 
 
 
 19.45 
 6 
 
 
 20.45 
 5 
 
 
 20.3 
 5 
 
 
 75.3 
 4 
 
 
 44 
 4 
 
 
 19.3 
 4 
 
 
 20.4 
 4 
 
 
 49.9 
 4 
 
 
 20.35 
 4 
 
 
 19.85 
 4 
 
 
 50.15 
 4 
 
 
 70.6 
 4 
 
 
 74.7 
 4 
 
 
 69.9 
 4 
 
 
 19.4 
 4 
 
 
 69.6 
 4 
 
 
 20.5 
 4 
 
 
 19.2 
 4 
 
 
 44.4 
 4 
 
 
 69.65 
 4 
 
 
 19.5 
 4 
 
 
 69.95 
 4 
 
 
 19.95 
 4 
 
 
 2317.1 
 3 
 
 
 24.8 
 3 
 
 
 383.65 
 3 
 
 
 80.55 
 3 
 
 
 74.9 
 3 
 
 
 69.1 
 3 
 
 
 470.2 
 3 
 
 
 74.3 
 3 
 
 
 1284.2 
 3 
 
 
 55.7 
 3 
 
 
 70.3 
 3 
 
 
 24.4 
 3 
 
 
 69.55 
 3 
 
 
 84.5 
 3 
 
 
 220.45 
 3 
 
 
 20 
 3 
 
 
 74.6 
 3 
 
 
 50.45 
 3 
 
 
 50.6 
 3 
 
 
 50.75 
 3 
 
 
 79.55 
 3 
 
 
 19.25 
 3 
 
 
 75.35 
 3 
 
 
 85.5 
 3 
 
 
 85 
 3 
 
 
 69.25 
 3 
 
 
 35.9 
 3 
 
 
 25.25 
 3 
 
 
 70.1 
 3 
 
 
 20.1 
 3 
 
 
 45.1 
 3 
 
 
 86.05 
 3 
 
 
 70.45 
 3 
 
 
 70.15 
 3 
 
 
 20.55 
 3 
 
 
 19.1 
 3 
 
 
 74.35 
 3 
 
 
 45.85 
 3 
 
 
 44.75 
 3 
 
 
 20.9 
 3 
 
 
 45.7 
 3 
 
 
 305.55 
 3 
 
 
 71 
 2 
 
 
 259.8 
 2 
 
 
 145.15 
 2 
 
 
 906.85 
 2 
 
 
 467.15 
 2 
 
 
 854.9 
 2 
 
 
 3496.3 
 2 
 
 
 7334.05 
 2 
 
 
 3187.65 
 2 
 
 
 74.2 
 2 
 
 
 143.65 
 2 
 
 
 5714.2 
 2 
 
 
 4370.25 
 2 
 
 
 470.6 
 2 
 
 
 482.8 
 2 
 
 
 1235.55 
 2 
 
 
 24.7 
 2 
 
 
 268.45 
 2 
 
 
 85.1 
 2 
 
 
 94.4 
 2 
 
 
 178.7 
 2 
 
 
 926.25 
 2 
 
 
 522.95 
 2 
 
 
 1714.95 
 2 
 
 
 89.3 
 2 
 
 
 503.6 
 2 
 
 
 20.95 
 2 
 
 
 98.5 
 2 
 
 
 91.7 
 2 
 
 
 1782.4 
 2 
 
 
 78.65 
 2 
 
 
 25.7 
 2 
 
 
 3297 
 2 
 
 
 83.75 
 2 
 
 
 202.3 
 2 
 
 
 415.55 
 2 
 
 
 57.4 
 2 
 
 
 19.6 
 2 
 
 
 1110.05 
 2 
 
 
 134.05 
 2 
 
 
 3132.75 
 2 
 
 
 1077.5 
 2 
 
 
 729.95 
 2 
 
 
 96.8 
 2 
 
 
 246.7 
 2 
 
 
 260.9 
 2 
 
 
 727.8 
 2 
 
 
 316.9 
 2 
 
 
 84.6 
 2 
 
 
 5731.85 
 2 
 
 
 147.15 
 2 
 
 
 5682.25 
 2 
 
 
 193.6 
 2 
 
 
 70.85 
 2 
 
 
 777.3 
 2 
 
 
 300.4 
 2 
 
 
 1885.15 
 2 
 
 
 267.35 
 2 
 
 
 449.75 
 2 
 
 
 137.6 
 2 
 
 
 592.65 
 2 
 
 
 53.55 
 2 
 
 
 2640.55 
 2 
 
 
 511.25 
 2 
 
 
 74.45 
 2 
 
 
 1025.15 
 2 
 
 
 669.45 
 2 
 
 
 84.2 
 2 
 
 
 1139.2 
 2 
 
 
 181.65 
 2 
 
 
 280.85 
 2 
 
 
 63.75 
 2 
 
 
 76.35 
 2 
 
 
 348.8 
 2 
 
 
 307.4 
 2 
 
 
 44.45 
 2 
 
 
 1201.65 
 2 
 
 
 1873.7 
 2 
 
 
 477.55 
 2 
 
 
 493.4 
 2 
 
 
 86.6 
 2 
 
 
 498.1 
 2 
 
 
 262.05 
 2 
 
 
 2651.2 
 2 
 
 
 5969.3 
 2 
 
 
 416.45 
 2 
 
 
 44.15 
 2 
 
 
 70.25 
 2 
 
 
 2404.15 
 2 
 
 
 71.1 
 2 
 
 
 45.8 
 2 
 
 
 190.25 
 2 
 
 
 487.05 
 2 
 
 
 90.55 
 2 
 
 
 1008.7 
 2 
 
 
 288.05 
 2 
 
 
 70.05 
 2 
 
 
 601.6 
 2 
 
 
 2254.2 
 2 
 
 
 151.3 
 2 
 
 
 256.6 
 2 
 
 
 155.35 
 2 
 
 
 740.3 
 2 
 
 
 25.1 
 2 
 
 
 382.2 
 2 
 
 
 90.6 
 2 
 
 
 71.35 
 2 
 
 
 2661.1 
 2 
 
 
 505.95 
 2 
 
 
 1046.5 
 2 
 
 
 4976.15 
 2 
 
 
 1443.65 
 2 
 
 
 460.25 
 2 
 
 
 683.25 
 2 
 
 
 1672.35 
 2 
 
 
 45.15 
 2 
 
 
 161.45 
 2 
 
 
 318.6 
 2 
 
 
 1381.8 
 2 
 
 
 49.55 
 2 
 
 
 45.05 
 2 
 
 
 44.9 
 2 
 
 
 61.05 
 2 
 
 
 340.35 
 2 
 
 
 844.45 
 2 
 
 
 74.4 
 2 
 
 
 229.55 
 2 
 
 
 1305.95 
 2 
 
 
 202.25 
 2 
 
 
 69.35 
 2 
 
 
 113.5 
 2 
 
 
 475.1 
 2 
 
 
 49.8 
 2 
 
 
 529.5 
 2 
 
 
 168.15 
 2 
 
 
 1672.1 
 2 
 
 
 398.55 
 2 
 
 
 1493.2 
 2 
 
 
 294.45 
 2 
 
 
 196.9 
 2 
 
 
 469.65 
 2 
 
 
 587.1 
 2 
 
 
 50.55 
 2 
 
 
 238.5 
 2 
 
 
 2570.2 
 2 
 
 
 3418.2 
 2 
 
 
 25.05 
 2 
 
 
 355.1 
 2 
 
 
 235.05 
 2 
 
 
 5597.65 
 2 
 
 
 20.65 
 2 
 
 
 340.85 
 2 
 
 
 654.85 
 2 
 
 
 1183.05 
 2 
 
 
 184.1 
 2 
 
 
 4858.7 
 2 
 
 
 402.6 
 2 
 
 
 75.45 
 2 
 
 
 494.05 
 2 
 
 
 1642.75 
 2 
 
 
 184.4 
 2 
 
 


TotalCharges should not be a categotical Features. It should be converted to numerical feature

In [27]:
dfTelcoChurn['TotalCharges'] = pd.to_numeric(dfTelcoChurn['TotalCharges'], errors='coerce') 

In [28]:
dfTelcoChurn = dfTelcoChurn.replace(np.nan, 0, regex=True)

In [29]:
NumericFeatures = dfTelcoChurn.select_dtypes(include=['int64', 'float64']).columns
NumericFeatures

Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [30]:
CategoricalFeatures = dfTelcoChurn.select_dtypes(include='object').columns

#### Also we observed that the Categoric Features in Telco Churn Data is suitable for the On-Hot encoding.<br> 
#### The number of distict features are much less for the features. This was unlike the case in Adult Data

In [31]:
TelcoDict = {'Categoric': CategoricalFeatures}

In [32]:
dfTelcoChurn[NumericFeatures]

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges
0,0,1,29.85,29.85
1,0,34,56.95,1889.50
2,0,2,53.85,108.15
3,0,45,42.30,1840.75
4,0,2,70.70,151.65
5,0,8,99.65,820.50
6,0,22,89.10,1949.40
7,0,10,29.75,301.90
8,0,28,104.80,3046.05
9,0,62,56.15,3487.95


In [33]:
ColumnDict['Telecom'] ={'Numeric':NumericFeatures, 'Categoric': CategoricalFeatures}

In [34]:
ColumnDict

{'Adult': {'Numeric': Index(['age', 'fnlwgt', 'educational-num', 'capital-gain', 'capital-loss',
         'hours-per-week'],
        dtype='object'),
  'Categoric': Index(['workclass', 'education', 'marital-status', 'occupation',
         'relationship', 'race', 'gender', 'native-country', 'income'],
        dtype='object')},
 'Telecom': {'Numeric': Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object'),
  'Categoric': Index(['customerID', 'gender', 'Partner', 'Dependents', 'PhoneService',
         'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
         'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
         'Contract', 'PaperlessBilling', 'PaymentMethod', 'Churn'],
        dtype='object')}}

## Now to carry out the Modelling including the Encoding
We shall use Pipelines here

In [35]:
 EncodersList =[ ce.backward_difference.BackwardDifferenceEncoder, 
                 ce.basen.BaseNEncoder,
                 ce.binary.BinaryEncoder,
                 ce.cat_boost.CatBoostEncoder,
                 ce.hashing.HashingEncoder,
                 ce.helmert.HelmertEncoder,
                 ce.james_stein.JamesSteinEncoder,
                 ce.one_hot.OneHotEncoder,
                 ce.leave_one_out.LeaveOneOutEncoder,
                 ce.m_estimate.MEstimateEncoder,
                 ce.ordinal.OrdinalEncoder,
#                  ce.polynomial.PolynomialEncoder,
                 ce.sum_coding.SumEncoder,
                 ce.target_encoder.TargetEncoder,
                 ce.woe.WOEEncoder
                ]


In [36]:
len(EncodersList)

14

In [37]:
DatasetList = {"Telecom": {'data' :dfTelcoChurn, 'Target': 'Churn'}  ,
               "Adult" :{'data': dfAdultData, 'Target': 'income'} 
              }

In [38]:
def experimentEncoder (datasetName, expDataFrame, expTarget, expColumnList, expEncoderList) :
    print ("There are %d Features in Data Set [%s] " %( ( expDataFrame.shape[1]-1), datasetName) )
    #print ("Encoders to be applied %d" %len(expEncoderList) )
    #print ("Target Column is %s" %expTarget )
    
    X = expDataFrame.drop(expTarget, axis=1)
    y = expDataFrame[expTarget]
    
    le = preprocessing.LabelEncoder()
    #label_encoder = le.fit_transform(y) 
    label_encoder = le.fit(y)
    #print(label_encoder.classes_)
    y = label_encoder.transform(y)
    
    #print (y)
    # Split the dataset into Train and Test
    X_Train, X_Test, y_Train, y_Test = train_test_split (X,y, test_size=0.2)


    numericTransPipeline = Pipeline(steps=[
                    ('imputer', SimpleImputer(strategy='median')),
                    ('scaler', StandardScaler())]
                )
    
    RunParams = {'EncoderModel', 'SplitColumns', 'F1-Score'}
    RunResult = pd.DataFrame(columns = {'EncoderModel', 'SplitColumns', 'F1-Score'})
    
    #print(RunResult)

#     TX=[ ('num', numeric_transformer, expColumnList['Numeric'] )]
        
#     ct = ColumnTransformer (transformers=TX)
#     X_trainTransNumeric = ct.fit(X)
#     print(X_trainTransNumeric)

    #ser = pd.Series(expColumnList['Categoric'])
    CatFeatures = pd.Series(expColumnList['Categoric'])
    CatFeatures = CatFeatures[ 0:-1 ]

    #print(CatFeatures)


    #print (expColumnList['Categoric'])


    for encoder in expEncoderList :
        encoderName = str (object=encoder).split('.')[2][:-2]
        
        #print (encoderName)
        categoricalTransPipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('woe', encoder())])
        
        TX=[ ('num', numericTransPipeline, expColumnList['Numeric'] ), \
             ('cat', categoricalTransPipeline, CatFeatures)]
        
        preprocessor = ColumnTransformer (transformers=TX)
        X_trainTrans = preprocessor.fit_transform(X_Train, y_Train)
        # Main Outer pipeline to first preprocess and then build the prediction Model
        classifierPipe = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', RandomForestClassifier(n_estimators=500))])
        model = classifierPipe.fit(X_Train, y_Train)
        yHat = model.predict(X_Test)
        score = f1_score(y_Test, yHat, average='macro')
        #print(f1_score(y_test, y_pred, average='macro'))
        RunResult = RunResult.append({'EncoderModel': encoderName, 'SplitColumns' : X_trainTrans.shape[1], 'F1-Score': score}, ignore_index=True)
        #print(RunResult)
        #print("Number of Columns in [%s] Encoder is %d || Score [%f]" % (encoderName, X_trainTrans.shape[1], score) )
    
    #RunResult = RunResult[['EncoderModel', 'SplitColumns', 'F1-Score']]
    #print("End of subroutine")
    return RunResult


In [42]:
DatasetResults = pd.DataFrame (columns={'DatasetName', 'EncoderModel', 'SplitColumns', 'F1-Score' })
i=0
for dataKey in DatasetList.keys() :
    #print("To Invoke Experiment for %s" %dataKey)
    #print(ColumnDict.get(dataKey)['Categoric'])
    result = experimentEncoder(dataKey, DatasetList.get(dataKey)['data'],  DatasetList.get(dataKey)['Target'], ColumnDict.get(dataKey), EncodersList)
    #DatasetResults = DatasetResults.append({'DatasetName' : dataKey}, ignore_index=True)
    DatasetResults = DatasetResults.append(result, sort=True, ignore_index=True)
    print ("Value of i = {%d}",i)
    DatasetResults.loc[i: i+len(EncodersList), 'DatasetName'] = dataKey
    i=i+len(EncodersList)
    print("")

There are 20 Features in Data Set [Telecom] 


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Value of i = {%d} 0

There are 14 Features in Data Set [Adult] 


/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Value of i = {%d} 14



In [44]:
DatasetResults.sort_values(["DatasetName", "F1-Score"], ascending=False, inplace=True)

In [45]:
DatasetResults

,DatasetName,EncoderModel,F1-Score,SplitColumns
3,Telecom,CatBoostEncoder,0.737616,20
2,Telecom,BinaryEncoder,0.715713,58
1,Telecom,BaseNEncoder,0.713042,58
12,Telecom,TargetEncoder,0.709834,20
6,Telecom,JamesSteinEncoder,0.708481,20
13,Telecom,WOEEncoder,0.705204,20
11,Telecom,SumEncoder,0.704628,5664
9,Telecom,MEstimateEncoder,0.702841,20
7,Telecom,OneHotEncoder,0.698760,5679
10,Telecom,OrdinalEncoder,0.695407,20
